In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms, datasets

In [7]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [1]:
EPOCHS     = 300
BATCH_SIZE = 128

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data',
                    train=True,
                    download=True,
                    transform=transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))
                    ])),
    batch_size=BATCH_SIZE,(
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data',
                    train=False,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))
                    ])),
    batch_size=BATCH_SIZE,
    shuffle=True))

0it [00:00, ?it/s]

100%|█████████▉| 170221568/170498071 [01:06<00:00, 7955072.23it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


170500096it [01:20, 7955072.23it/s]                               

In [5]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                         kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    

In [6]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                              stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)
    
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [8]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [9]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [11]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [12]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct =0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # batch loss sum
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # highest value index is prediction value
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS+1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

/home/rrbb/.local/lib/python3.5/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] Test Loss: 1.2667, Accuracy: 53.89%
[2] Test Loss: 1.1369, Accuracy: 59.46%
[3] Test Loss: 1.0580, Accuracy: 65.18%
[4] Test Loss: 0.9413, Accuracy: 69.10%
[5] Test Loss: 0.6958, Accuracy: 75.70%
[6] Test Loss: 0.9177, Accuracy: 70.08%
[7] Test Loss: 1.5411, Accuracy: 61.29%
[8] Test Loss: 0.7566, Accuracy: 74.63%
[9] Test Loss: 0.6084, Accuracy: 79.36%
[10] Test Loss: 1.4560, Accuracy: 61.27%
[11] Test Loss: 1.1150, Accuracy: 65.41%
[12] Test Loss: 0.6534, Accuracy: 77.38%
[13] Test Loss: 0.6454, Accuracy: 78.26%
[14] Test Loss: 0.7478, Accuracy: 76.18%
[15] Test Loss: 0.8065, Accuracy: 72.36%
[16] Test Loss: 0.9671, Accuracy: 70.13%
[17] Test Loss: 0.7470, Accuracy: 75.39%
[18] Test Loss: 0.6386, Accuracy: 78.57%
[19] Test Loss: 0.6391, Accuracy: 78.14%
[20] Test Loss: 0.6967, Accuracy: 75.87%
[21] Test Loss: 0.6917, Accuracy: 77.00%
[22] Test Loss: 0.6872, Accuracy: 78.05%
[23] Test Loss: 0.6691, Accuracy: 78.13%
[24] Test Loss: 0.7132, Accuracy: 76.96%
[25] Test Loss: 0.8626, A